### Importing libraries

In [16]:
import openravepy
import trajoptpy
import json
import numpy as np
import trajoptpy.kin_utils as ku
from trajoptpy.check_traj import traj_is_safe
import humanoidspy
import time
import random 
import matplotlib.pyplot as plt
import pickle
from trajopt_util import *
from planning_util import *
from regression import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

FILENAME = 'data/atlas_planning/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Setting the trajopt

In [17]:
np.set_printoptions(precision=4, suppress=True)

env = openravepy.Environment()
env.StopSimulation()
env.Load("../bigdata/atlas.xml")
env.Load("../env/bookshelves.env.xml")
env.SetDefaultViewer()

True

### Setting the robot

In [18]:
robot = env.GetRobots()[0]        
init_transform = np.eye(4)
init_transform[:3,3] = [-.35, 1, .92712]
init_transform[:3,3] = [-0.15, 0., .92712]
#init_transform[:3,3] = [2.6, 1, .92712]
robot.SetTransform(init_transform)

robot.SetDOFValues([-1.3],[robot.GetJoint("l_arm_shx").GetDOFIndex()])
robot.SetDOFValues([1.3],[robot.GetJoint("r_arm_shx").GetDOFIndex()])
standing_posture = robot.GetActiveDOFValues()

dof = robot.GetActiveDOF()

In [19]:
def request_skeleton(n_steps):
    request = {
        "basic_info" : {
            "n_steps" : n_steps,
            "manip" : "active",
            "start_fixed" : True
        },
        "costs" : [
        {
            "type" : "joint_vel",            
            "params": {"coeffs":([.1]*14 + [.05]*14)}
        },
        {
            "type":"collision",
            "params":{"coeffs":[2], "dist_pen":[.005], "continuous":False}
        },
        {
            "type":"collision","name":"cont_coll",
            "params":{"coeffs":[2], "dist_pen":[.005], "continuous":True}
        }
        ],
        "constraints" : [
        ],
        "init_info" : {
            "type" : "stationary"
        }
    }
    for i in xrange(1,n_steps):
        request["costs"].extend([
         #{
             #"type":"potential_energy",
             #"params":{"coeff" : .0005,"timestep":i}
         #},
        #{
            #"type":"static_torque",
            #"params":{"coeff" : .01,"timestep":i}
        #}                    
        ])
    return request  

def press_button_request(robot, hand_xyz, hand_link, foot_links, n_steps):
    """
    Sets up the problem to safely shift the weight to the other foot (to_foot)
    Suppose to_foot = "r_foot"    
    Then problem constrains both feet to remain at fixed poses (their current poses)
    at all intermediate timesteps, the center of mass lies over the convex hull of l_foot and r_foot
    at the final timestep, the center of mass lies over r_foot
    """    
    
    from_foot, to_foot = foot_links
    
    request = request_skeleton(n_steps)
    from_foot_xyz, from_foot_quat = xyzQuatFromMatrix(robot.GetLink(from_foot).GetTransform())
    to_foot_xyz, to_foot_quat = xyzQuatFromMatrix(robot.GetLink(to_foot).GetTransform())
    
    for i in xrange(1, n_steps):
        request["constraints"].extend([
            {
                "type":"pose",
                "name":"from_foot_pose",
                "params":{
                    "xyz":list(from_foot_xyz),
                    "wxyz":list(from_foot_quat),
                    "link":from_foot,
                    "timestep":i
                }
            },
            {
                "type":"pose",
                "name":"to_foot_pose",
                "params":{
                    "xyz":list(to_foot_xyz),
                    "wxyz":list(to_foot_quat),
                    "link":to_foot,
                    "timestep":i
                }
            }
        ])    
        request["constraints"].append(
            {
                "type":"zmp","name":"zmp_%i"%i,
                "params":{"planted_links":[from_foot, to_foot],"timestep":i}
            })
    request["constraints"].append(
        {
            "type":"pose",
            "name":"final_hand_pose",
            "params":{
                "xyz":list(hand_xyz),
                "wxyz":[0.5*np.sqrt(2),0,0,0.5*np.sqrt(2)],
                "link":hand_link,
                "pos_coeffs":[1,1,1],
                "rot_coeffs":[0.0,0.0,0.0],
                "timestep":i
            }
        }        
    )

    return request

In [20]:
def check_traj_atlas(result, target, threshold = 0.01):
    traj = result.GetTraj()
    #check for collision
    if traj_is_safe(traj, robot) is not True:
        print "There is a collision within the trajectory!"
        return False

    #check target for pose constraints
    robot.SetActiveDOFValues(traj[-1])
    xyz = robot.GetLink('r_hand').GetTransform()[0:3,3]
    if (np.linalg.norm(xyz - target) > threshold):
        print('Target is not reached!')
        return False

    print 'Optimization is success!'
    return True

In [21]:
def generate_valid_xyz(limits):
    is_success = False

    while is_success is not True:
        x,y,z = generate_random_xyz(limits)
        #create a box and check for collision
        is_col = check_col_with_box(env,x,y,z)
        if is_col:
            #print('There is collision!')
            continue
        else:
            #print('No collision!')
            is_success = True
    
    return x,y,z

def xyzQuatFromMatrix(T):
    wxyz_xyz = openravepy.poseFromMatrix(T)
    return wxyz_xyz[4:7], wxyz_xyz[0:4]

### Build / Retrieve Dataset

In [22]:
def build_database(x_init, y_init, num_traj, limits, n_steps = 30,predictor=None):
    num_plan = 0
    comp_times = []
    costs = []
    tic = time.time()
    
    while len(x_init) < num_traj:
        print('{}th Planning, {} trajectories in database'.format(num_plan,len(x_init)))
        #Get a random pose as target
        init_joint = standing_posture.copy()
        robot.SetActiveDOFValues(init_joint)
        xyz_target = generate_valid_xyz(limits)
        robot.SetActiveDOFValues(init_joint)

        
        request_standard = press_button_request(robot, xyz_target, "r_hand", ["l_foot","r_foot"],n_steps)
        duration, result = run_opt(request_standard, env)
   
        #Check traj result
        traj = result.GetTraj()
        if check_traj_atlas(result, xyz_target):
            print 'Planning is successfull!'
            x_init.append(xyz_target)
            y_init.append(traj.flatten())
            comp_times.append(duration)
            costs.append(result.GetCosts()[0][1])
        else:
            print('Fail to find good solution!') 

        num_plan += 1
        
        
    #Store the result
    toc = time.time()
    total_time = toc-tic
    success_rate = num_traj*1.0/num_plan
    x_init = np.vstack(x_init)
    y_init = np.vstack(y_init)
    
    
    data = dict()
    data['x'] = x_init
    data['y'] = y_init
    data['total_time'] = total_time
    data['success_rate'] = success_rate
    data['comp_times'] = comp_times
    data['costs'] = costs
    return data

In [23]:
to_build_database = False
to_augment_database = True
num_traj = 1
n_steps = 10
limits = [[0.3, 0.7],[-0.5, 0.5],[0.8,1.4]]

if to_build_database:
    if to_augment_database:
        #load data
        data = pickle.load( open(FILENAME + 'data.pkl', 'rb'))
        x_init = list(data['x'])
        y_init = list(data['y'])
        num_traj += len(x_init)     
    else:
        x_init,y_init = [],[]
        
    data = build_database(x_init, y_init, num_traj,limits, n_steps = n_steps)
    x_init = data['x']
    y_init = data['y']
    pickle.dump(data,open(FILENAME + 'data.pkl', 'wb') )
    print('Success_rate : {}, average costs:{}'.format(data['success_rate'], np.mean(data['costs'])))
else:
    #load data
    data = pickle.load( open(FILENAME + 'data.pkl', 'rb'))
    x_init = data['x']
    y_init = data['y']
    num_traj = len(x_init)

### Reduce the data sample

In [24]:
x_init = x_init[0:1000]
y_init = y_init[0:1000]

### Apply Dimensionality Reduction to the data

In [25]:
from sklearn.decomposition import PCA
y_pca = PCA(n_components = 50)
y_pca.fit(y_init)
y_init_reduced = y_pca.transform(y_init)

### Build Function Approximator

In [ ]:
dim_input = len(x_init[0])
is_load_regressor = False

nn = NN_Regressor()
gpy = GPy_Regressor(dim_input = dim_input)
gpy_pca = GPy_Regressor(dim_input = dim_input, is_transform='PCA')
dpglm = DP_GLM_Regressor()
dpglm_pca = DP_GLM_Regressor(is_transform='PCA')

if is_load_regressor:
    nn.load_from_file(FILENAME + 'nn')
    gpy.load_from_file(FILENAME + 'gpy')
    gpy_pca.load_from_file(FILENAME + 'gpy_pca')
    dpglm.load_from_file(FILENAME + 'dpglm')
    dpglm_pca.load_from_file(FILENAME + 'dpglm_pca')
else:
    print 'Planning for NN'
    nn.fit(x_init, y_init)
    nn.save_to_file(FILENAME + 'nn')

    print 'Planning for GPY'
    gpy.fit(x_init, y_init, num_restarts=10)
    gpy.save_to_file(FILENAME + 'gpy')
    
    print 'Planning for GPY PCA'
    gpy_pca.fit(x_init, y_init_reduced)
    gpy_pca.save_to_file(FILENAME + 'gpy_pca')

    
    print 'Planning for DPGLM'
    dpglm.fit(x_init,y_init)
    dpglm.save_to_file(FILENAME + 'dpglm')

    print 'Planning for DPGLM PCA'
    dpglm_pca.fit(x_init,y_init_reduced)
    dpglm_pca.save_to_file(FILENAME + 'dpglm_pca')

### Testing the warmstart

In [ ]:
#creating the test cases
create_test = True
if create_test:
    x_test = []
    target_joints = []
    num_test = 300
    for i in range(num_test):
        init_joint = standing_posture.copy()
        robot.SetActiveDOFValues(init_joint)
        xyz_target = generate_valid_xyz(limits)
        robot.SetActiveDOFValues(init_joint)
        cur_x = xyz_target
        x_test.append(cur_x)

    x_test = np.vstack(x_test)
    data_test = dict()
    data_test['x_test'] = x_test
    test_file = open(FILENAME + 'data_test.npy', 'wb')
    pickle.dump( data_test,test_file)
    test_file.close()
else:
    #load data
    test_file = open(FILENAME + 'data_test.npy', 'rb')
    data_test = pickle.load(test_file)
    x_test = data_test['x_test']
    num_test = len(x_test)
    

In [ ]:
def set_init(request, init_type, init_values):
    request["init_info"]["type"] = "given_traj"

    if init_type == 'straight':
        request['init_info']['endpoint'] = init_values.tolist()
    elif init_type == 'given_traj':
        request["init_info"]["data"] = [row.tolist() for row in init_values]
    else:
        print('Initialization {} is not defined'.format(init_type))
        return None
    return request

In [ ]:
method_names = ['STD    ',  'NN     ', 'GPY    ', 'BGMR   ', 'GPY_PCA   ', 'BGMR_PCA  ']
methods = [nn, gpy, dpglm, gpy_pca, dpglm_pca]
results = dict()
result,success, duration = dict(),dict(),dict()
for method in method_names:
    results[method] = dict()
    results[method]['costs'] = []
    results[method]['successes'] = []
    results[method]['comp_times'] = []
    results[method]['func_evals'] = []
    results[method]['qp_solves'] = []
    
num_test = 250
ik_times = []
quat = [1,0,0,0]
for i in range(num_test):
    print('{}th Planning'.format(i))
    
    #setting up the problem case
    #index = np.random.randint(0,len(x_test))
    index = i
    x_cur = x_test[index:index+1,:]
    xyz_target = x_cur.flatten()
    robot.SetActiveDOFValues(init_joint)
    
    #without warmstart
    method_name = method_names[0]
    request_standard = press_button_request(robot, xyz_target, "r_hand", ["l_foot","r_foot"],n_steps)
    duration[method_name], result[method_name] = run_opt(request_standard, env)
    success[method_name] = check_traj_atlas(result[method_name],xyz_target)
    
    
    #Other warmstart methods
    for i,method in enumerate(methods):
        method_name = method_names[i+1]
        if isinstance(method, Straight_Regressor):
            traj,cov = method.predict(init_joint0, target_joint)
        elif isinstance(method, DP_GLM_Regressor):
            traj,cov = method.predict(x_cur, return_gmm = True)
        else:
            traj,cov = method.predict(x_cur)
            
        if method.is_transform == 'PCA':
            traj = y_pca.inverse_transform(traj)
        traj = traj.reshape(-1,dof)
        traj[0,:] = init_joint
        robot.SetActiveDOFValues(init_joint)
        request_traj = press_button_request(robot, xyz_target, "r_hand", ["l_foot","r_foot"],n_steps)
        request_traj = set_init(request_traj, 'given_traj', traj)
        duration[method_name], result[method_name] = run_opt(request_traj, env)    
        success[method_name] = check_traj_atlas(result[method_name],xyz_target)
    #Record the result
    for method_name in method_names:
        results[method_name]['costs'].append(result[method_name].GetCosts()[0][1])
        results[method_name]['func_evals'].append(result[method_name].GetNumFuncEval()[0])
        results[method_name]['qp_solves'].append(result[method_name].GetNumQPSolve()[0])
        results[method_name]['successes'].append(success[method_name])      
        results[method_name]['comp_times'].append(duration[method_name])


### Printing Results

In [ ]:
print(' Method \t| Success Rate \t| Conv. Time \t| Traj. Cost \t| Func. Evals \t| QP Solves')
for method in method_names:
    successes = np.array(results[method]['successes'])
    success = np.count_nonzero(successes)
    
    comp_times = np.array(results[method]['comp_times'])[successes]
    costs = np.array(results[method]['costs'])[successes]
    func_evals = np.array(results[method]['func_evals'])[successes]
    qp_solves = np.array(results[method]['qp_solves'])[successes]
    
    print('{0}: \t& {1:.3f} \t& {2:.2f} \t& {3:.3f}  \\\\'.format(method, success*1.0/len(successes), np.sum(comp_times)/success, np.sum(costs)/success))
    

### Printing Results Tied to Standard Success

In [ ]:
print(' Method \t| Success Rate \t| Conv. Time \t| Traj. Cost \t| Func. Evals \t| QP Solves')
standard_successes = np.array(results['STD    ']['successes'])

for method in method_names:
    successes = np.array(results[method]['successes'])[standard_successes]
    success = np.count_nonzero(successes)
    comp_times = np.array(results[method]['comp_times'])[standard_successes][successes]
    costs = np.array(results[method]['costs'])[standard_successes][successes]
    func_evals = np.array(results[method]['func_evals'])[standard_successes][successes]
    qp_solves = np.array(results[method]['qp_solves'])[standard_successes][successes]
    
    print('{0}: \t {1:.3f} \t {2:.2f} \t {3:.3f} \t {4:.3f} \t {5:.3f}'.format(method, success*1.0/len(successes), np.sum(comp_times)/success, np.sum(costs)/success, 1.0*np.sum(func_evals)/success, 1.0*np.sum(qp_solves)/success ))
    